# Notebook to estimate stress/threat to GDEs from human activities

Determine whether GDEs are at risk of damage from people
- Roads and housing density - redundant?
- Surface water diversions from NDWR database - PODs?

## Set up environments in ArcGIS Pro

In [ ]:
# Import ArcGIS modules and check out spatial analyst extension
import arcpy
import os
import pandas as pd
import numpy as np
from arcpy import env
from arcpy.sa import *
arcpy.CheckOutExtension("spatial")

In [ ]:
# Path to temporary geodatabase
#path =  r"folder_path\gdbname.gdb"
path =  r"K:\GIS3\Projects\GDE\Maps\GDE_Threats\GDE_Threats.gdb"

# Environment settings
env.workspace = path
env.overwriteOutput = True
env.outputCoordinateSystem = arcpy.SpatialReference(26911) # Spatial reference NAD 1983 UTM Zone 11N. The code is '26911'

In [ ]:
# GDE layers
# springs = r'path_to_folder\NV_iGDE_assess.gdb\Springs'
# wet = r'path_to_folder\NV_iGDE_assess.gdb\Wetlands'
# phr = r'path_to_folder\NV_iGDE_assess.gdb\Phreatophytes_Explode'
# lp = r'path_to_folder\NV_iGDE_assess.gdb\Lakes_Playas'
# rs = r'path_to_folder\NV_iGDE_assess.gdb\Rivers_Streams'

springs = r'E:\RCF Data Recovery\Recovered Files\P00\GDE_Threats\Hydrology\NV_iGDE_assess.gdb\Springs'
wet = r'E:\RCF Data Recovery\Recovered Files\P00\GDE_Threats\Hydrology\NV_iGDE_assess.gdb\Wetlands'
phr = r'E:\RCF Data Recovery\Recovered Files\P00\GDE_Threats\Hydrology\NV_iGDE_assess.gdb\Phreatophytes_Explode'
lp = r'E:\RCF Data Recovery\Recovered Files\P00\GDE_Threats\Hydrology\NV_iGDE_assess.gdb\Lakes_Playas'
rs = r'E:\RCF Data Recovery\Recovered Files\P00\GDE_Threats\Hydrology\NV_iGDE_assess.gdb\Rivers_Streams'

## Surface Diversions and Stress to GDEs

Surface diversion data (POD/point of diversion) from NDWR. Buffer by half-mile.

Attribute: __SurfaceDiv = 0.1 (least stress due to uncertainty in data) or 1 (surface diversion observed, stressed)__

In [ ]:
# Subset surface diversions from POD dataset
pod = arcpy.CopyFeatures_management(r'path_to_folder\POD_101719.shp', 'pods_surface')
print([f.name for f in arcpy.ListFields(pod)])
print(arcpy.GetCount_management(pod))

In [ ]:
# Subset types of diversions
surface = ['STO', 'STR', 'SPR', 'LAK', 'RES', 'OSW'] # storage, stream, spring, lake, rservoir, other surface water
with arcpy.da.UpdateCursor(pod, 'source') as cursor:
    for row in cursor:
        if (row[0] not in surface):
            print(row[0])
            cursor.deleteRow()
del cursor
print(arcpy.GetCount_management(pod))

In [ ]:
# Same as water level trend data - buffer by a half-mile and overlay GDEs
pod_buff = arcpy.Buffer_analysis(pod, 'pods_surface_halfmile', '0.5 Mile', '', '', 'ALL')
pod_buff = r'pods_surface_halfmile'
print([f.name for f in arcpy.ListFields(pod_buff)])

In [ ]:
# Select and attribute GDEs, as above
# Springs
arcpy.AddField_management(springs, 'SurfaceDiv', 'DOUBLE')
arcpy.CalculateField_management(springs, 'SurfaceDiv', 0.1)
[f.name for f in arcpy.ListFields(springs)]
spr_select = arcpy.SelectLayerByLocation_management(springs, 'INTERSECT', pod_buff)
arcpy.CalculateField_management(spr_select, 'SurfaceDiv', 1)

# Phreatophytes
arcpy.AddField_management(phr, 'SurfaceDiv', 'DOUBLE')
arcpy.CalculateField_management(phr, 'SurfaceDiv', 0.1)
[f.name for f in arcpy.ListFields(phr)]
phr_select = arcpy.SelectLayerByLocation_management(phr, 'INTERSECT', pod_buff)
arcpy.CalculateField_management(phr_select, 'SurfaceDiv', 1)

# Wetlands
arcpy.AddField_management(wet, 'SurfaceDiv', 'DOUBLE')
arcpy.CalculateField_management(wet, 'SurfaceDiv', 0.1)
[f.name for f in arcpy.ListFields(wet)]
wet_select = arcpy.SelectLayerByLocation_management(wet, 'INTERSECT', pod_buff)
arcpy.CalculateField_management(wet_select, 'SurfaceDiv', 1)

# Rivers/Streams
arcpy.AddField_management(rs, 'SurfaceDiv', 'DOUBLE')
arcpy.CalculateField_management(rs, 'SurfaceDiv', 0.1)
[f.name for f in arcpy.ListFields(rs)]
rs_select = arcpy.SelectLayerByLocation_management(rs, 'INTERSECT', pod_buff)
arcpy.CalculateField_management(rs_select, 'SurfaceDiv', 1)

# Lakes/Playas
arcpy.AddField_management(lp, 'SurfaceDiv', 'DOUBLE')
arcpy.CalculateField_management(lp, 'SurfaceDiv', 0.1)
[f.name for f in arcpy.ListFields(lp)]
lp_select = arcpy.SelectLayerByLocation_management(lp, 'INTERSECT', pod_buff)
arcpy.CalculateField_management(lp_select, 'SurfaceDiv', 1)


## Format housing density data

Read in rasters of current (2010) and future (2060) housing density. Claculate change in housing density for threat analysis.


In [ ]:
# Housing Density
hd_now = arcpy.Raster(r'path_to_folder\density_theobald_mos_2010.tif')
hd_future = arcpy.Raster(r'path_to_folder\density_theobald_2060.tif')

# Normalize 0 to 1
hd_now_norm = hd_now/hd_now.maximum
hd_future_norm = hd_future/hd_future.maximum

# Save normalized rasters
arcpy.CopyRaster_management(hd_now_norm, r'aoth_to_folder\density_theobald_2010_norm.tif')
arcpy.CopyRaster_management(hd_future_norm, r'path_to_folder\density_theobald_2060_norm.tif')


In [ ]:
# Change in housing density
hd_change = hd_future - hd_now 
arcpy.CopyRaster_management(hd_change, r'path_to_folder\density_theobald_2010to2060.tif')

# Positive values == increase in housing density
# Make negative values (places that decreased in density) NULL
x = SetNull(hd_change, hd_change, "Value < 1")
x.save('density_change_null')
hd_change_norm = x/x.maximum
arcpy.CopyRaster_management(hd_change_norm, r'path_to_folder\density_theobald_change_norm.tif')


## Stress from current housing density

Not using roads for this value; too redundant.

Attribute: __HousingStr = 0 to 1 (less to more stress depending on normalized housing density)__

In [ ]:
# Current housing density
hd = arcpy.Raster(r'path_to_folder\density_theobald_2010_norm.tif')

In [ ]:
# Springs
temp_spr = arcpy.sa.ExtractValuesToPoints(springs, hd, 'springs_house_dens')
[f.name for f in arcpy.ListFields(temp_spr)]
arcpy.AddField_management(temp_spr, 'HousingStr', 'DOUBLE')
arcpy.CalculateField_management(temp_spr, 'HousingStr', '!RASTERVALU!', 'PYTHON3')
arcpy.JoinField_management(springs, 'OBJECTID', temp_spr, 'OBJECTID', ['HousingStr'])
[f.name for f in arcpy.ListFields(springs)]

# Wetlands
temp_wet = arcpy.sa.ZonalStatisticsAsTable(wet, 'OBJECTID', hd, 'wetlands_house_dens_tbl', 'DATA', 'MEAN')
[f.name for f in arcpy.ListFields(temp_wet)]
arcpy.AddField_management(temp_wet, 'HousingStr', 'DOUBLE')
arcpy.CalculateField_management(temp_wet, 'HousingStr', '!MEAN!', 'PYTHON3')
arcpy.JoinField_management(wet, 'OBJECTID', temp_wet, 'OBJECTID_1', ['HousingStr'])
[f.name for f in arcpy.ListFields(wet)]

# PHreatophytes
temp_phr = arcpy.sa.ZonalStatisticsAsTable(phr, 'OBJECTID', hd, 'phr_house_dens_tbl', 'DATA', 'MEAN')
[f.name for f in arcpy.ListFields(temp_phr)]
arcpy.AddField_management(temp_phr, 'HousingStr', 'DOUBLE')
arcpy.CalculateField_management(temp_phr, 'HousingStr', '!MEAN!', 'PYTHON3')
arcpy.JoinField_management(phr, 'OBJECTID', temp_phr, 'OBJECTID_1', ['HousingStr'])
[f.name for f in arcpy.ListFields(phr)]

# Lakes/playas
temp_lp = arcpy.sa.ZonalStatisticsAsTable(lp, 'OBJECTID', hd, 'lp_house_dens_tbl', 'DATA', 'MEAN')
[f.name for f in arcpy.ListFields(temp_lp)]
arcpy.AddField_management(temp_lp, 'HousingStr', 'DOUBLE')
arcpy.CalculateField_management(temp_lp, 'HousingStr', '!MEAN!', 'PYTHON3')
arcpy.JoinField_management(lp, 'OBJECTID', temp_lp, 'OBJECTID_1', ['HousingStr'])
[f.name for f in arcpy.ListFields(lp)]

# Rivers/streams
temp_rs = arcpy.sa.ZonalStatisticsAsTable(rs, 'OBJECTID', hd, 'rs_house_dens_tbl', 'DATA', 'MEAN')
[f.name for f in arcpy.ListFields(temp_rs)]
arcpy.AddField_management(temp_rs, 'HousingStr', 'DOUBLE')
arcpy.CalculateField_management(temp_rs, 'HousingStr', '!MEAN!', 'PYTHON3')
arcpy.JoinField_management(rs, 'OBJECTID', temp_rs, 'OBJECTID_1', ['HousingStr'])
[f.name for f in arcpy.ListFields(rs)]

## Threat from projected change in housing density

Attribute: __HousingThr = 0 to 1 (less to more stress depending on normalized housing density change)__

In [ ]:
# Change in projected housing density
hd = r'path_to_folder\density_theobald_change_norm.tif'

In [ ]:
# Springs
arcpy.DeleteField_management(springs, 'HousingThr')
temp_spr = arcpy.sa.ExtractValuesToPoints(springs, hd, 'springs_house_dens_change')
[f.name for f in arcpy.ListFields(temp_spr)]
arcpy.AddField_management(temp_spr, 'HousingThr', 'DOUBLE')
arcpy.CalculateField_management(temp_spr, 'HousingThr', '!RASTERVALU!', 'PYTHON3')
arcpy.JoinField_management(springs, 'OBJECTID', temp_spr, 'OBJECTID', ['HousingThr'])
[f.name for f in arcpy.ListFields(springs)]

# Wetlands
arcpy.DeleteField_management(wet, 'HousingThr')
temp_wet = arcpy.sa.ZonalStatisticsAsTable(wet, 'OBJECTID', hd, 'wetlands_house_denschange_tbl', 'DATA', 'MEAN')
[f.name for f in arcpy.ListFields(temp_wet)]
arcpy.AddField_management(temp_wet, 'HousingThr', 'DOUBLE')
arcpy.CalculateField_management(temp_wet, 'HousingThr', '!MEAN!', 'PYTHON3')
arcpy.JoinField_management(wet, 'OBJECTID', temp_wet, 'OBJECTID_1', ['HousingThr'])
[f.name for f in arcpy.ListFields(wet)]

# PHreatophytes
arcpy.DeleteField_management(phr, 'HousingThr')
temp_phr = arcpy.sa.ZonalStatisticsAsTable(phr, 'OBJECTID', hd, 'phr_house_denschange_tbl', 'DATA', 'MEAN')
[f.name for f in arcpy.ListFields(temp_phr)]
arcpy.AddField_management(temp_phr, 'HousingThr', 'DOUBLE')
arcpy.CalculateField_management(temp_phr, 'HousingThr', '!MEAN!', 'PYTHON3')
arcpy.JoinField_management(phr, 'OBJECTID', temp_phr, 'OBJECTID_1', ['HousingThr'])
[f.name for f in arcpy.ListFields(phr)]

# Lakes/playas
arcpy.DeleteField_management(lp, 'HousingThr')
temp_lp = arcpy.sa.ZonalStatisticsAsTable(lp, 'OBJECTID', hd, 'lp_house_denschange_tbl', 'DATA', 'MEAN')
[f.name for f in arcpy.ListFields(temp_lp)]
arcpy.AddField_management(temp_lp, 'HousingThr', 'DOUBLE')
arcpy.CalculateField_management(temp_lp, 'HousingThr', '!MEAN!', 'PYTHON3')
arcpy.JoinField_management(lp, 'OBJECTID', temp_lp, 'OBJECTID_1', ['HousingThr'])
[f.name for f in arcpy.ListFields(lp)]

# Rivers/streams
arcpy.DeleteField_management(rs, 'HousingThr')
temp_rs = arcpy.sa.ZonalStatisticsAsTable(rs, 'OBJECTID', hd, 'rs_house_denschange_tbl', 'DATA', 'MEAN')
[f.name for f in arcpy.ListFields(temp_rs)]
arcpy.AddField_management(temp_rs, 'HousingThr', 'DOUBLE')
arcpy.CalculateField_management(temp_rs, 'HousingThr', '!MEAN!', 'PYTHON3')
arcpy.JoinField_management(rs, 'OBJECTID', temp_rs, 'OBJECTID_1', ['HousingThr'])
[f.name for f in arcpy.ListFields(rs)]
